In [1]:
import qkeras as qk
import tensorflow as tf
import tensorflow.keras as k

2024-03-16 15:58:54.320622: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-16 15:58:54.320652: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-16 15:58:54.321517: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-16 15:58:54.326789: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [24]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model5 = tf.keras.models.Sequential([
tf.keras.layers.Input((28,28,1), name="input"),
qk.QConv2D(16, 3, input_shape=(28, 28, 1), kernel_quantizer=qk.ternary(alpha="auto", use_stochastic_rounding=True)),
qk.QActivation("quantized_relu(2,0)"),
qk.QConv2D(16, 3, input_shape=(28, 28, 1), kernel_quantizer=qk.ternary(alpha="auto", use_stochastic_rounding=True)),
qk.QActivation("quantized_relu(2,0)"),
tf.keras.layers.MaxPooling2D(strides=2),
tf.keras.layers.Flatten(),
qk.QDense(10, kernel_quantizer=qk.ternary(alpha="auto", use_stochastic_rounding=True),
           bias_quantizer=qk.ternary(alpha="auto", use_stochastic_rounding=True))])
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model5.compile(optimizer='rmsprop', loss=loss_fn, metrics=['accuracy'])
model5.fit(x_train.reshape(60000, 28, 28, 1), y_train, epochs=6)

Epoch 1/6
1875/1875 [==============================] - 25s 12ms/step - loss: 0.2538 - accuracy: 0.9264
Epoch 2/6
1875/1875 [==============================] - 24s 13ms/step - loss: 0.1021 - accuracy: 0.9697
Epoch 3/6
1875/1875 [==============================] - 24s 13ms/step - loss: 0.0820 - accuracy: 0.9752
Epoch 4/6
1875/1875 [==============================] - 24s 13ms/step - loss: 0.0773 - accuracy: 0.9768
Epoch 5/6
1875/1875 [==============================] - 25s 13ms/step - loss: 0.0669 - accuracy: 0.9796
Epoch 6/6
1875/1875 [==============================] - 25s 13ms/step - loss: 0.0633 - accuracy: 0.9804


In [29]:
qk.print_qstats(model5)

/home/metal/dev/misc/python/lib/python3.11/site-packages/keras/src/constraints.py:365: UserWarning: The `keras.constraints.serialize()` API should only be used for objects of type `keras.constraints.Constraint`. Found an instance of type <class 'qkeras.quantizers.ternary'>, which may lead to improper serialization.
  warnings.warn(


Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.



Number of operations in model:
    q_conv2d_14                   : 97344 (smux_2_8)
    q_conv2d_15                   : 1327104 (smux_2_2)
    q_dense_5                     : 23040 (smux_2_2)

Number of operation types in model:
    smux_2_2                      : 1350144
    smux_2_8                      : 97344

Weight profiling:
    q_conv2d_14_weights            : 144   (2-bit unit)
    q_conv2d_14_bias               : 16    (32-bit unit)
    q_conv2d_15_weights            : 2304  (2-bit unit)
    q_conv2d_15_bias               : 16    (32-bit unit)
    q_dense_5_weights              : 23040 (2-bit unit)
    q_dense_5_bias                 : 10    (2-bit unit)

Weight sparsity:
... quantizing model
    q_conv2d_14                    : 0.2938
    q_conv2d_15                    : 0.5534
    q_dense_5                      : 0.4908
    ----------------------------------------
    Total Sparsity                 : 0.4952


In [30]:
qk.utils.quantized_model_dump(model5, x_train[:10], layers_to_dump=["q_conv2d_14", "q_conv2d_15", "q_dense_5"])

temp dir /tmp/tmpw6qbiazy
create dir /tmp/tmpw6qbiazy
1/1 [==============================] - 1s 658ms/step
writing the layer output tensor to  /tmp/tmpw6qbiazy/q_conv2d_14.bin
writing the layer output tensor to  /tmp/tmpw6qbiazy/q_conv2d_15.bin
writing the layer output tensor to  /tmp/tmpw6qbiazy/q_dense_5.bin


In [31]:
model5.save("model5-ternary.keras")

/home/metal/dev/misc/python/lib/python3.11/site-packages/keras/src/constraints.py:365: UserWarning: The `keras.constraints.serialize()` API should only be used for objects of type `keras.constraints.Constraint`. Found an instance of type <class 'qkeras.quantizers.ternary'>, which may lead to improper serialization.
  warnings.warn(


In [32]:
tf.keras.models.save_model(model5, "ternary_saved")

INFO:tensorflow:Assets written to: ternary_saved/assets


INFO:tensorflow:Assets written to: ternary_saved/assets
/home/metal/dev/misc/python/lib/python3.11/site-packages/keras/src/constraints.py:365: UserWarning: The `keras.constraints.serialize()` API should only be used for objects of type `keras.constraints.Constraint`. Found an instance of type <class 'qkeras.quantizers.ternary'>, which may lead to improper serialization.
  warnings.warn(


In [33]:
json_config = model5.to_json()

/home/metal/dev/misc/python/lib/python3.11/site-packages/keras/src/constraints.py:365: UserWarning: The `keras.constraints.serialize()` API should only be used for objects of type `keras.constraints.Constraint`. Found an instance of type <class 'qkeras.quantizers.ternary'>, which may lead to improper serialization.
  warnings.warn(


In [34]:
json_config

'{"class_name": "Sequential", "config": {"name": "sequential_6", "layers": [{"module": "keras.layers", "class_name": "InputLayer", "config": {"batch_input_shape": [null, 28, 28, 1], "dtype": "float32", "sparse": false, "ragged": false, "name": "input"}, "registered_name": null}, {"module": "qkeras.qconvolutional", "class_name": "QConv2D", "config": {"name": "q_conv2d_14", "trainable": true, "dtype": "float32", "batch_input_shape": [null, 28, 28, 1], "filters": 16, "kernel_size": [3, 3], "strides": [1, 1], "padding": "valid", "data_format": "channels_last", "dilation_rate": [1, 1], "groups": 1, "activation": "linear", "use_bias": true, "kernel_initializer": {"module": "keras.initializers", "class_name": "HeNormal", "config": {"seed": null}, "registered_name": null}, "bias_initializer": {"module": "keras.initializers", "class_name": "Zeros", "config": {}, "registered_name": null}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": {"m

In [37]:
cc = model5.get_layer('q_conv2d_14')

In [46]:
with tf.session() as sess:
    cc.kernel_quantizer.scale.eval(sess)

AttributeError: module 'tensorflow' has no attribute 'session'